# Import Drive to use the uploaded data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)
curDir =('/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project')
!cat /content/gdrive/My\ Drive/Colab\ Notebooks/DeepMovement_project

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
cat: '/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project': Is a directory


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Setup the needed dependancies

In [4]:
import os
from IPython.display import clear_output
os.chdir(curDir)
!pwd
# !cd /content/gdrive/My\ Drive/Colab\ Notebooks/Pose\ project
!git clone https://www.github.com/ildoonet/tf-openpose
os.chdir('./tf-openpose')
!pip install -r requirements.txt
os.chdir(curDir)
clear_output

/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project
fatal: destination path 'tf-openpose' already exists and is not an empty directory.
  Cloning https://github.com/ppwwyyxx/tensorpack.git to /tmp/pip-req-build-3ebmyo4p
  Running command git clone -q https://github.com/ppwwyyxx/tensorpack.git /tmp/pip-req-build-3ebmyo4p
  Created wheel for tensorpack: filename=tensorpack-0.9.8-py2.py3-none-any.whl size=291857 sha256=2917f459058fb7a53cb17c8177e81322a0e45020811d0d299df1084d0d5b574b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgerjtjf/wheels/a8/b5/a9/025b3a1294b9ffff93309e6956c65aa80e0fa40821d29eff1e
Successfully built tensorpack


<function IPython.core.display.clear_output>

In [5]:
os.chdir(curDir)
!apt-get install swig
os.chdir('./tf-openpose/tf_pose/pafprocess')
!pwd
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
clear_output
os.chdir(curDir+'/tf-openpose')
!python setup.py install
clear_output
os.chdir(curDir)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project/tf-openpose/tf_pose/pafprocess
running build_ext
/content/gdrive/My: Operation not supported
running install
running bdist_egg
running egg_info
writing tf_pose.egg-info/PKG-INFO
writing dependency_links to tf_pose.egg-info/dependency_links.txt
writing requirements to tf_pose.egg-info/requires.txt
writing top-level names to tf_pose.egg-info/top_level.txt
file pafprocess.py (for module pafprocess) not found
writing manifest file 'tf_pose.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
file pafprocess.py (for module pafprocess) not found
copying tf_pose/pafprocess/pafprocess.py -> build/lib.linux-x86_64-3.6/tf_pose/pafprocess
file pafprocess.py (for module paf

# Code Beginning

In [6]:
# In[1]
import pandas as pd
import numpy as np
import pickle
import cv2
import os
import argparse
import logging
import sys
import time
from sklearn.preprocessing import MinMaxScaler
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from keras.models import load_model, model_from_json
# from google.colab import files
# src = list(files.upload().values())[0]
# open('UsedFunctions.py','wb').write(src)
os.chdir(curDir)
from UsedFunctions import *
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Using TensorFlow backend.


In [0]:
# In[2]
### Reading and processing the data for decision
def Regulate(framesInSeq):
    rem=[]
    for j in range(len(framesInSeq)):
        if framesInSeq[j] <= 7:
            if len(framesInSeq) <= 1:
                print("Error, very small file")
                return None
            if j == 0 and (j+1) < len(framesInSeq):
                framesInSeq[j+1] += framesInSeq[j]
            elif j == len(framesInSeq)-1 and (j-1) >= 0:
                framesInSeq[j-1] += framesInSeq[j]
            else:
                if framesInSeq[j+1] < framesInSeq[j-1]:
                    framesInSeq[j+1] += framesInSeq[j]  
                else: framesInSeq[j-1] += framesInSeq[j]
            rem.append(j)
    framesInSeq=np.delete(framesInSeq,rem)
    return framesInSeq if framesInSeq.tolist() else None

def ReadData(data, seq=50):
    df = pd.concat(data)  #read data
    
    #editing on data
    df.dropna(axis=0, how='any', thresh=76, subset=None, inplace=True)
    
    fr = df.copy()['Frame']
    #dropping usless columns and columns with all their values are 0
    dropped = ['Human', 'V_title', 'Frame', 'L Foot_x_PCM', 'L Foot_y_PCM', 'R Foot_L',	'L Foot_L',	'Head & Neck_R',	'Trunk_R',	'R Upper Arm_R',	'L Upper Arm_R',	'R Forearm_R',	'L Forearm_R',	'R Hand_R',	'L Hand_R',	'R Thigh_R',	'L Thigh_R',	'R Shank_R',	'L Shank_R',	'R Foot_R',	'L Foot_R']
    df.drop(columns=dropped, inplace=True)
    
    seq_count = np.zeros(fr.shape, dtype=int)
    for i in range(1, len(fr.index)):
        # assuming if there is > 30 frames gap, the scene is changed
        if np.abs(fr.values[i-1] - fr.values[i]) > 30:
            seq_count[i] = seq_count[i-1] + 1
        else:
            seq_count[i] = seq_count[i-1]
    framesInSeq = np.bincount(seq_count)
    
    framesInSeq = Regulate(framesInSeq)
    if framesInSeq is None:
        return None
    nan_locations = np.isnan(np.float_(df.values)) # get locations of nan values to set to 0 later
    df.fillna(0, inplace=True) # convert NaN values to zeros
    
    array = df.values #conver df to array
    X = np.array(array, dtype='float')
    
    #feature scaling 
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X[nan_locations]=0 # set location of nans to 0
    
    Data_X = []
    for cLen in framesInSeq:
        for i in range((cLen // seq) + 1 * ((cLen % seq)!=0)):
            x = []
            for j in range(seq):
                if (seq*i + j) >= cLen:
                    x = np.tile(x, (seq//len(x)+1, 1))[:seq].tolist()
                    break
                else:
                    x.append(list(X[seq*i + j]))
            Data_X.append(x)
    return np.array(Data_X, dtype='float32')

In [0]:
# In[3]

# logger = logging.getLogger('TfPoseEstimator')
# logger.setLevel(logging.DEBUG)
# ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
# formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
# ch.setFormatter(formatter)
# logger.addHandler(ch)

In [0]:
# In[4]
### Init
""" write your own path here """
video_path = './Actions_set/running/person12_running_d2_uncomp.avi'#'./samples/P1/2001.mp4'

resolution = '432x368'
poseModel = 'mobilenet_thin'
show_process = False
resize = '0x0'
showBG = True
resize_out_ratio = 4.0

### Adding argument variables
# parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
# parser.add_argument('--video-path', type=str, default=video_path,
#                     help='if provided, select videos in that directory. ')
# parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
# parser.add_argument('--poseModel', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin')
# parser.add_argument('--show-process', type=bool, default=False,
#                     help='for debug purpose, if enabled, speed for inference is dropped.')
# parser.add_argument('--resize', type=str, default='0x0',
#                     help='if provided, resize images before they are processed. default=0x0, Recommends : 432x368 or 656x368 or 1312x736 ')
# parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')
# parser.add_argument('--resize-out-ratio', type=float, default=4.0,
#                     help='if provided, resize heatmaps before they are post-processed. default=1.0')
# #parser.add_argument('--mode', type=str, default='test',
# #                    help='if provided, choose the run mode to be eather train or test. default=test ')
# args = parser.parse_args()


## Capture Video frames

In [9]:
# In[5]
### Capturing the video frames
beg = time.time()             # For time computing

# logger.debug('initialization %s : %s' % (poseModel, get_graph_path(poseModel)))

w, h = model_wh(resize)
if w == 0 or h == 0:
	e = TfPoseEstimator(get_graph_path(poseModel), target_size=(432, 368))
else:
	e = TfPoseEstimator(get_graph_path(poseModel), target_size=(w, h))
"""        
newpath=os.getcwd()+'/Output'+'/'
if not os.path.exists(newpath):
    os.makedirs(newpath)
OUTPUT_PATH=newpath+'/'+'Humans_'+mode+'.csv'
"""

OUTPUTS = []

video_timing=time.time()             # For time computing
print("Reading video file ", video_path)

cap = cv2.VideoCapture(video_path)
ret_val = True
if cap.isOpened() is False:
    print("Error opening video stream or file")
count = 0
human_prev = None #@
while ret_val: # True as long as there are frames
    
    begin_counting=time.time()             # For time computing
    ret_val, image = cap.read()
    if not ret_val:
        break
    print ("Frame No: " , count)
    count += 1
    if count%2:
        print('skipping frame ')
        continue
    # Saving Frame
    #start=time.time()
    #cv2.imwrite("Output/"+h_name+"_Frame_No_"+str(count)+".jpg",image)
    #print('Time taken for writing a Frame is {:.3f} ms'.format((time.time()-start)*1000))
    
    if image is None:
        logger.error('Image can not be read, path=%s' % video_path)
        sys.exit(-1)
    
    humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
    #humans = e.inference(image)
    print (humans)
    if len(humans)==0 or len(humans)>1:
        print("Inconvenient frame")
        continue

    Co_ordinates=manipulate(humans, human_prev, frame=count-1, mode='test')     # Computes the Co-ordinates from the given data by O-Nect
    human_prev = Co_ordinates.iloc[0].copy()#@
    
    #print('Time taken for Co-ordinate calculations for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-begin_counting)*1000))
    
    # Extracting X, Y Coordinates
    #start=time.time()
    X_Coords, Y_Coords= Get_Coords(Co_ordinates)
    #print('Time taken for Co-ordinate extractions for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))
    
    # Adding more features based on X, Y Coordinates
    #start=time.time()
    PCM_Frames= Calculate_PCM(X_Coords, Y_Coords)
    #print('Time taken for PCM for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
    
    #start=time.time()
    TCM_x, TCM_y= Calculate_TCM(PCM_Frames)
    #print('Time taken for TCM for {} Frames is {:.3f} ms'.format(len(TCM_x),(time.time()-start)*1000))
    
    #start=time.time()
    L= Calculate_L(TCM_x, TCM_y, PCM_Frames)
    #print('Time taken for L features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
    
    #start=time.time()
    D1, D2, D3 = Calculate_D(PCM_Frames, TCM_x, TCM_y, 'Degrees')
    #print('Time taken for D1, D2, D3 features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
    
    #start=time.time()
    R=Calculate_R(PCM_Frames)
    #print('Time taken for R feature for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))
    
    #start=time.time()
    out=Add_Features_To_dataframe(Co_ordinates, PCM_Frames, TCM_x, TCM_y, L, R, D1, D2, D3)
    #print('Time taken for adding features to dataframe for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))
    
    print('Time taken for the whole file of {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-begin_counting)*1000))
    OUTPUTS.append(out)
    clear_output()

Frame No:  250
skipping frame 
Frame No:  251
[]
Inconvenient frame
Frame No:  252
skipping frame 
Frame No:  253
[]
Inconvenient frame
Frame No:  254
skipping frame 
Frame No:  255
[]
Inconvenient frame
Frame No:  256
skipping frame 
Frame No:  257
[]
Inconvenient frame
Frame No:  258
skipping frame 
Frame No:  259
[]
Inconvenient frame
Frame No:  260
skipping frame 
Frame No:  261
[]
Inconvenient frame
Frame No:  262
skipping frame 
Frame No:  263
[]
Inconvenient frame
Frame No:  264
skipping frame 
Frame No:  265
[]
Inconvenient frame
Frame No:  266
skipping frame 
Frame No:  267
[]
Inconvenient frame
Frame No:  268
skipping frame 
Frame No:  269
[]
Inconvenient frame
Frame No:  270
skipping frame 
Frame No:  271
[]
Inconvenient frame
Frame No:  272
skipping frame 
Frame No:  273
[]
Inconvenient frame
Frame No:  274
skipping frame 
Frame No:  275
[]
Inconvenient frame
Frame No:  276
skipping frame 
Frame No:  277
[]
Inconvenient frame
Frame No:  278
skipping frame 
Frame No:  279
[]

## Model Reading

In [10]:
# In[6]
### Prediction

# load the mutual model variables from disk
modelspath=os.path.join(curDir, 'ModelData', 'Activity_P1')
filename = os.path.join(modelspath, 'LSTM_model_variables.sav')
with open(filename, 'rb') as handle:
    classes = pickle.load(handle)
    sequence_size = pickle.load(handle)
    batch_size = pickle.load(handle)
    lr = pickle.load(handle)
    decay = pickle.load(handle)
    lstm_out = pickle.load(handle)
    dense = pickle.load(handle)
#sequence_size = 10
#batch_size = 64

# loading the model
#lstm_out = [128, 256, 512, 1024, 64]
#dense_layer = 128
#model = modelBuild(Data_X.shape[1:], lstm_out, [dense_layer, len(classes)])

with open(os.path.join(modelspath,'model_architecture.json'), 'r') as f:
    model = model_from_json(f.read())
print(model.summary()) # optional


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-10-09 13:50:58,399 WARNING From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 79)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           106496    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 64)            49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387 

In [11]:
df = pd.concat(OUTPUTS)  #read data

#editing on data
df.dropna(axis=0, how='any', thresh=76, subset=None, inplace=True)

fr = df.copy()['Frame']
#dropping usless columns and columns with all their values are 0
dropped = ['Human', 'V_title', 'Frame', 'L Foot_x_PCM', 'L Foot_y_PCM', 'R Foot_L',	'L Foot_L',	'Head & Neck_R',	'Trunk_R',	'R Upper Arm_R',	'L Upper Arm_R',	'R Forearm_R',	'L Forearm_R',	'R Hand_R',	'L Hand_R',	'R Thigh_R',	'L Thigh_R',	'R Shank_R',	'L Shank_R',	'R Foot_R',	'L Foot_R']
df.drop(columns=dropped, inplace=True)

seq_count = np.zeros(fr.shape, dtype=int)
for i in range(1, len(fr.index)):
    # assuming if there is > 30 frames gap, the scene is changed
    if np.abs(fr.values[i-1] - fr.values[i]) > 30:
        seq_count[i] = seq_count[i-1] + 1
    else:
        seq_count[i] = seq_count[i-1]
framesInSeq = np.bincount(seq_count)
framesInSeq = Regulate(framesInSeq)
not framesInSeq.tolist()

AttributeError: ignored

In [0]:
# Reading data 
# sequence_size = 50 ##@ Change later
Data_X= ReadData(OUTPUTS, sequence_size)
if Data_X is None:
    print('Skip this video')
else: print(Data_X.shape)

Skip this video


## Test Results

In [0]:
# In[7]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Loading weights of the trained model 
# chose = os.path.join(modelspath,'Chosen')
# for f in os.listdir(chose):
model.load_weights(os.path.join(modelspath, 'my_model_weights.hdf5'))
#model.load_weights(os.path.join(modelspath,'my_model_weights.hdf5'))
# print('for weights in %s' %f)
predicted = model.predict(Data_X, batch_size=batch_size).argmax(axis=1)
print('Predicted value is ', predicted)
print('The predicted person is most likly to be:')
Ps = set(predicted)
n = Ps
for f in range(len(Ps)):
    m = max(n, key=list(predicted).count)
    print('{}\twith probability {:.2f}%'.format(classes[m], list(predicted).count(m)/len(predicted)*100))
    n.remove(m)
print('Total time taken is {:.3f} s'.format((time.time()-beg)))


Predicted value is  [1 1 1 1]
The predicted person is most likly to be:
running	with probability 100.00%
Total time taken is 1264.997 s


#Other Tests

## Combined Method

In [0]:
# In[5]
### Capturing the video frames
beg = time.time()             # For time computing
# logger.debug('initialization %s : %s' % (poseModel, get_graph_path(poseModel)))

video_path = './Actions_set'

resolution = '432x368'
poseModel = 'mobilenet_thin'
show_process = False
resize = '0x0'
showBG = True
resize_out_ratio = 4.0

w, h = model_wh(resize)
if w == 0 or h == 0:
	e = TfPoseEstimator(get_graph_path(poseModel), target_size=(432, 368))
else:
	e = TfPoseEstimator(get_graph_path(poseModel), target_size=(w, h))
"""        
newpath=os.getcwd()+'/Output'+'/'
if not os.path.exists(newpath):
    os.makedirs(newpath)
OUTPUT_PATH=newpath+'/'+'Humans_'+mode+'.csv'
"""
valid_videos = [".avi",".mp4",".mov",".mkv",".rmvb", ".dv", ".ts"] 
O =[]
F = []
cnt = 0
right = 0
for D in os.listdir(video_path):
    for f in os.listdir(os.path.join(video_path, D)):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_videos:
            continue
        OUTPUTS = []

        video_timing=time.time()             # For time computing
        print("Reading video file ", f)

        cap = cv2.VideoCapture(os.path.join(video_path, D, f))
        ret_val = True
        if cap.isOpened() is False:
            print("Error opening video stream or file")
        count = 0
        human_prev = None #@
        while ret_val: # True as long as there are frames

            begin_counting=time.time()             # For time computing
            ret_val, image = cap.read()
            if not ret_val:
                break
            #print ("Frame No: " , count)
            count += 1
            if count%2:
                #print('skipping frame ')
                continue
            # Saving Frame
            #start=time.time()
            #cv2.imwrite("Output/"+h_name+"_Frame_No_"+str(count)+".jpg",image)
            #print('Time taken for writing a Frame is {:.3f} ms'.format((time.time()-start)*1000))

            if image is None:
                logger.error('Image can not be read, path=%s' % video_path)
                sys.exit(-1)

            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
            #humans = e.inference(image)
            #print (humans)
            if len(humans)==0 or len(humans)>1:
                #print("Inconvenient frame")
                continue

            Co_ordinates=manipulate(humans, human_prev, frame=count-1, mode='test')     # Computes the Co-ordinates from the given data by O-Nect
            human_prev = Co_ordinates.iloc[0].copy()#@

            #print('Time taken for Co-ordinate calculations for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-begin_counting)*1000))

            # Extracting X, Y Coordinates
            #start=time.time()
            X_Coords, Y_Coords= Get_Coords(Co_ordinates)
            #print('Time taken for Co-ordinate extractions for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))

            # Adding more features based on X, Y Coordinates
            #start=time.time()
            PCM_Frames= Calculate_PCM(X_Coords, Y_Coords)
            #print('Time taken for PCM for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))

            #start=time.time()
            TCM_x, TCM_y= Calculate_TCM(PCM_Frames)
            #print('Time taken for TCM for {} Frames is {:.3f} ms'.format(len(TCM_x),(time.time()-start)*1000))

            #start=time.time()
            L= Calculate_L(TCM_x, TCM_y, PCM_Frames)
            #print('Time taken for L features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))

            #start=time.time()
            D1, D2, D3 = Calculate_D(PCM_Frames, TCM_x, TCM_y, 'Degrees')
            #print('Time taken for D1, D2, D3 features for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))

            #start=time.time()
            R=Calculate_R(PCM_Frames)
            #print('Time taken for R feature for {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-start)*1000))

            #start=time.time()
            out=Add_Features_To_dataframe(Co_ordinates, PCM_Frames, TCM_x, TCM_y, L, R, D1, D2, D3)
            #print('Time taken for adding features to dataframe for {} Frames is {:.3f} ms'.format(len(Co_ordinates),(time.time()-start)*1000))

            #print('Time taken for the whole file of {} Frames is {:.3f} ms'.format(len(PCM_Frames),(time.time()-begin_counting)*1000))
            OUTPUTS.append(out)
        
        Data_X= ReadData(OUTPUTS, sequence_size)
        if Data_X is None:
            print('Skipping this video')
            continue
        print(Data_X.shape)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Loading weights of the trained model 
        # chose = os.path.join(modelspath,'Chosen')
        # for f in os.listdir(chose):
        model.load_weights(os.path.join(modelspath, 'my_model_weights.hdf5'))
        #model.load_weights(os.path.join(modelspath,'my_model_weights.hdf5'))
        # print('for weights in %s' %f)
        predicted = model.predict(Data_X, batch_size=batch_size).argmax(axis=1)
        print('Predicted value is ', predicted)
        print('The predicted person is most likly to be:')
        Ps = set(predicted)
        n = Ps
        for prop in range(len(Ps)):
            m = max(n, key=list(predicted).count)
            print('{}\twith probability {:.2f}%'.format(classes[m], list(predicted).count(m)/len(predicted)*100))
            if prop == 0:
                TruePred = classes[m]
            n.remove(m)
        print()
        cnt += 1
        if TruePred == D:
            right += 1
        O.append(OUTPUTS)
        F.append(str(str(D)+'/'+str(f)))
        # In[6]
        ### Prediction
TotAccuracy = (right / cnt) * 100
print('Total accuracy for all the files is %.4f%%' %(TotAccuracy))
# # load the mutual model variables from disk
# modelspath=os.path.join(os.getcwd(),'ModelData')
# filename = os.path.join(modelspath, 'LSTM_model_variables.sav')
# with open(filename, 'rb') as handle:
#     classes = pickle.load(handle)
#     sequence_size = pickle.load(handle)
#     batch_size = pickle.load(handle)
# # loading the model
# #lstm_out = [128, 256, 512, 1024, 64]
# #dense_layer = 128
# #model = modelBuild(Data_X.shape[1:], lstm_out, [dense_layer, len(classes)])

# with open(os.path.join(modelspath,'model_architecture.json'), 'r') as fmod:
#     Model = model_from_json(fmod.read())
# #         print(model.summary()) # optional
# # In[7]
# Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # Loading weights of the trained model 
# #         chose = os.path.join(modelspath,'Chosen')
# #         for f in os.listdir(chose):
# #sequence_size = 10
# #batch_size = 64

print('Total time taken is {:.3f} s'.format((time.time()-beg)))


[2019-10-09 13:51:45,951] [TfPoseEstimator] [INFO] loading graph from /usr/local/lib/python3.6/dist-packages/tf_pose-0.1.1-py3.6-linux-x86_64.egg/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2019-10-09 13:51:45,951 INFO loading graph from /usr/local/lib/python3.6/dist-packages/tf_pose-0.1.1-py3.6-linux-x86_64.egg/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=432x368)


## Prediction with different weight files

In [0]:
chose = os.path.join(modelspath,'500_500')
for Wt in os.listdir(chose):
    ext = os.path.splitext(Wt)[1]
    if ext.lower() != '.hdf5':
        continue
    # Reading data 

    Model.load_weights(os.path.join(chose,Wt))
    #model.load_weights(os.path.join(chose,'my_model_weights.hdf5'))

    for i in range(len(O)):
        Data_X= ReadData(O[i], sequence_size)
        predicted = Model.predict(Data_X, batch_size=batch_size).argmax(axis=1)
        print('for person in %s using weight file %s' %(F[i], Wt))
        print('Predicted value is ', predicted)
        print('The predicted person is most likly to be:')
        Ps = set(predicted)
        n = Ps
        for f in range(len(Ps)):
            m = max(n, key=list(predicted).count)
            print('{}\twith probability {:.2f}%'.format(classes[m], list(predicted).count(m)/len(predicted)*100))
            n.remove(m)

### PlayGround

In [0]:
for TT in os.listdir(os.path.join(modelspath,'Chosen')):
  print(TT)
clear_output()
print(TT)
print(os.listdir(video_path+'/P1'))
import glob
print(glob.glob(video_path+'/P1/*.mp4'))

In [0]:
print(O)